In [1]:
from __future__ import absolute_import, division, print_function,unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from six.moves  import urllib

import tensorflow.compat.v2.feature_column as fc
# fc used for linear regression 
import tensorflow as tf
from IPython.display import clear_output

In [2]:
CSV_COL_NAMES = ['SeptalLength','SeptalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Vesricolor','Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    'iris_training.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv'
)
test_path = tf.keras.utils.get_file(
    'iris_test.csv','https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv'
)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [4]:
train = pd.read_csv(train_path, names=CSV_COL_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COL_NAMES, header=0)

In [5]:
train.head()

,SeptalLength,SeptalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
trainy = train.pop('Species')
testy = test.pop('Species')

In [7]:
testy.head()
print(test.head())

   SeptalLength  SeptalWidth  PetalLength  PetalWidth
0           5.9          3.0          4.2         1.5
1           6.9          3.1          5.4         2.1
2           5.1          3.3          1.7         0.5
3           6.0          3.4          4.5         1.6
4           5.5          2.5          4.0         1.3


In [10]:
print(train.shape)
print(train.keys())

(120, 4)
Index(['SeptalLength', 'SeptalWidth', 'PetalLength', 'PetalWidth'], dtype='object')


In [9]:
def input_fn(features, labels, training=True, batch_size=256):
  # convert the inputs into datasets
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

  # shuffle and repeat if you are in the training mode:
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return(dataset.batch(batch_size))

In [13]:
my_features_col = []
for key in train.keys():
  my_features_col.append(tf.feature_column.numeric_column(key=key))
print(my_features_col)

[NumericColumn(key='SeptalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SeptalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [16]:
classifier = tf.estimator.DNNClassifier(
   feature_columns =  my_features_col,
  # selecting two hidden layers of 30 and 10 units respectively
  hidden_units=[30,10],
  
  # chosing 3 classes for this cuz there are 3 species of classes
  n_classes=3 
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpritd_fyu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
classifier.train(
    input_fn = lambda : input_fn(train, trainy, training=True),
    steps = 5000    
)

# using lambda to returnt the object of the input_fn like we did last time, input_fn object is returned with train, trainy, trainnig = true 
# steps = 5000 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpritd_fyu/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmpritd_fyu/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.3500399, step = 5000
INFO:tensorflow:global_step/sec: 456.358
INFO:tensorflow:loss = 0.34304112, step = 5100 (0.223 sec)
INFO:tensorflow:global_step/sec: 591.781
INFO:tensorflow:loss = 0.34493297, step = 5200 (0.169 sec)
INFO:tensorflow:global_step/sec: 587.712
INFO:tensorflow:loss = 0.3385268, step = 5300 (0.170 sec)
INFO:tensorflow:global_step/sec: 596.4
I

In [22]:
eval_result = classifier.evaluate(input_fn=lambda : input_fn(train, trainy, training=False))
print('\n the accuracy of the model is:',eval_result['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-11T09:11:47Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpritd_fyu/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20192s
INFO:tensorflow:Finished evaluation at 2021-01-11-09:11:48
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.94166666, average_loss = 0.34538454, global_step = 5000, loss = 0.34538454
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpritd_fyu/model.ckpt-5000

 the accuracy of the model is: 0.94166666


In [38]:
def input_fn(features, batch_size=256):
  # convert the input to a dataset without labels
  return(tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size))

features = ['SeptalLength','SeptalWidth','PetalLength','PetalWidth']
predict = {}

print('Type the numeric values as prompted')
for features in features:
  valid = True
  while valid:
    val = input(features + ":")
    if not val.isdigit(): valid = False
  predict[features] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for predict in predictions:
  class_id = predict['class_ids'][0]
  probabilities = predict['probabilities'][class_id]

  print(SPECIES[class_id],probabilities)
  print("prediction is '{}' {:.1f}%".format(
      SPECIES[class_id], 100*probabilities
  ))

  

Type the numeric values as prompted
SeptalLength:2.3
SeptalWidth:2.3
PetalLength:6
PetalLength:6.5
PetalWidth:7.6
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpritd_fyu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Virginica 0.9721985
prediction is 'Virginica' (97.2%


In [40]:
print(predict)

{'logits': array([-7.29964   ,  0.30415526,  3.8591232 ], dtype=float32), 'probabilities': array([1.3853686e-05, 2.7787663e-02, 9.7219849e-01], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}


In [44]:
predict_test = classifier.predict(input_fn= lambda : input_fn(test))
for predict, actual_label in zip(predict_test, testy):
  class_id = predict['class_ids'][0]
  probabilities = predict['probabilities'][class_id]

  # print(SPECIES[class_id],probabilities)
  print("prediction is '{}' {:.1f}% actual label is {}".format(
      SPECIES[class_id], 100*probabilities, SPECIES[actual_label]
  ))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpritd_fyu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
prediction is 'Vesricolor' 68.5% actual label is Vesricolor
prediction is 'Virginica' 69.4% actual label is Virginica
prediction is 'Setosa' 89.5% actual label is Setosa
prediction is 'Vesricolor' 64.7% actual label is Vesricolor
prediction is 'Vesricolor' 63.6% actual label is Vesricolor
prediction is 'Vesricolor' 75.2% actual label is Vesricolor
prediction is 'Setosa' 96.0% actual label is Setosa
prediction is 'Virginica' 58.4% actual label is Virginica
prediction is 'Vesricolor' 70.2% actual label is Vesricolor
prediction is 'Virginica' 85.4% actual label is Virginica
prediction is 'Virginica' 81.9% actual label is Virginica
prediction is 'Setosa' 92.2% actual label is Setosa
prediction is 'Virginica' 82.1% actual labe